<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Sparse_BIC_15_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (sparse)

In [21]:
# Function to generate weaker random CPDs per node
def generate_cpds_with_noise_and_variability():
    # Generate probabilities for each node independently
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()

    # Weaken dependencies by making conditional influence closer to uniform
    irt_given_ir_probs = np.random.rand(3, 3) * 0.5 + 0.25
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    ei_given_ir_irt_probs = np.random.rand(3, 3, 3) * 0.5 + 0.25
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    ue_given_geo_probs = np.random.rand(3, 3) * 0.5 + 0.25
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    ms_given_ei_geo_probs = np.random.rand(3, 3, 3) * 0.5 + 0.25
    ms_given_ei_geo_probs /= ms_given_ei_geo_probs.sum(axis=0, keepdims=True)

    irp_given_ir_irt_probs = np.random.rand(3, 3, 3) * 0.5 + 0.25
    irp_given_ir_irt_probs /= irp_given_ir_irt_probs.sum(axis=0, keepdims=True)

    inf_given_ms_irp_probs = np.random.rand(3, 3, 3) * 0.5 + 0.25
    inf_given_ms_irp_probs /= inf_given_ms_irp_probs.sum(axis=0, keepdims=True)

    # GDP, INV, CI nodes
    gdp_probs = np.random.rand(3)
    gdp_probs /= gdp_probs.sum()

    inv_probs = np.random.rand(3)
    inv_probs /= inv_probs.sum()

    ci_probs = np.random.rand(3)
    ci_probs /= ci_probs.sum()

    # New nodes: OIL, FX, IRB
    oil_probs = np.random.rand(3)
    oil_probs /= oil_probs.sum()

    fx_probs = np.random.rand(3)
    fx_probs /= fx_probs.sum()

    irb_probs = np.random.rand(3)
    irb_probs /= irb_probs.sum()

    # SP placeholder, updated dynamically per selected influencers
    sp_probs = np.random.rand(3, 3, 3, 3)  # Placeholder shape for SP, adjusted per influencers
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs, \
           ms_given_ei_geo_probs, irp_given_ir_irt_probs, inf_given_ms_irp_probs, gdp_probs, \
           inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, sp_probs

# Function to generate samples with varying dependencies and noise
def generate_and_save_samples_with_varying_dependencies(sample_size, filename):
    output_data = []

    # List of all nodes potentially influencing SP
    potential_influencers = ['IR', 'IRT', 'EI', 'GEO', 'UE', 'MS', 'IRP', 'INF', 'GDP', 'INV', 'CI', 'OIL', 'FX', 'IRB']

    for i in range(sample_size):
        # Generate CPDs for each entry
        ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, ms_probs, \
        irp_probs, inf_probs, gdp_probs, inv_probs, ci_probs, oil_probs, \
        fx_probs, irb_probs, sp_probs = generate_cpds_with_noise_and_variability()

        # Select 2-3 random nodes to influence SP for this entry
        num_influencers = np.random.choice([2, 3])
        selected_influencers = np.random.choice(potential_influencers, num_influencers, replace=False)

        # Initialize SP's probability distribution based on selected influencing nodes
        sp_probs_shape = [3] * (num_influencers + 1)  # Shape includes SP + selected influencers
        sp_probs = np.random.rand(*sp_probs_shape)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)

        # Sample each node's state individually
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]

        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]

        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]

        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]

        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]

        ms_probs_given_ei_geo = ms_probs[:, ei_state_idx, geo_state_idx]
        ms_state_idx = np.random.choice(3, p=ms_probs_given_ei_geo)
        ms_state = ['negative', 'neutral', 'positive'][ms_state_idx]

        irp_probs_given_ir_irt = irp_probs[:, ir_state_idx, irt_state_idx]
        irp_state_idx = np.random.choice(3, p=irp_probs_given_ir_irt)
        irp_state = ['loose', 'neutral', 'tight'][irp_state_idx]

        inf_probs_given_ms_irp = inf_probs[:, ms_state_idx, irp_state_idx]
        inf_state_idx = np.random.choice(3, p=inf_probs_given_ms_irp)
        inf_state = ['deflation', 'stable', 'inflation'][inf_state_idx]

        gdp_state_idx = np.random.choice(3, p=gdp_probs)
        gdp_state = ['low', 'medium', 'high'][gdp_state_idx]

        inv_state_idx = np.random.choice(3, p=inv_probs)
        inv_state = ['decreasing', 'steady', 'increasing'][inv_state_idx]

        ci_state_idx = np.random.choice(3, p=ci_probs)
        ci_state = ['low', 'medium', 'high'][ci_state_idx]

        oil_state_idx = np.random.choice(3, p=oil_probs)
        oil_state = ['low', 'medium', 'high'][oil_state_idx]

        fx_state_idx = np.random.choice(3, p=fx_probs)
        fx_state = ['weak', 'stable', 'strong'][fx_state_idx]

        irb_state_idx = np.random.choice(3, p=irb_probs)
        irb_state = ['loose', 'neutral', 'tight'][irb_state_idx]

        # Only use the state indices for the selected influencers to calculate SP probabilities
        sp_index = []
        for influencer in selected_influencers:
            if influencer == 'IR':
                sp_index.append(ir_state_idx)
            elif influencer == 'IRT':
                sp_index.append(irt_state_idx)
            elif influencer == 'EI':
                sp_index.append(ei_state_idx)
            elif influencer == 'GEO':
                sp_index.append(geo_state_idx)
            elif influencer == 'UE':
                sp_index.append(ue_state_idx)
            elif influencer == 'MS':
                sp_index.append(ms_state_idx)
            elif influencer == 'IRP':
                sp_index.append(irp_state_idx)
            elif influencer == 'INF':
                sp_index.append(inf_state_idx)
            elif influencer == 'GDP':
                sp_index.append(gdp_state_idx)
            elif influencer == 'INV':
                sp_index.append(inv_state_idx)
            elif influencer == 'CI':
                sp_index.append(ci_state_idx)
            elif influencer == 'OIL':
                sp_index.append(oil_state_idx)
            elif influencer == 'FX':
                sp_index.append(fx_state_idx)
            elif influencer == 'IRB':
                sp_index.append(irb_state_idx)

        # Use the selected influencers to determine SP's state
        sp_probs_selected = sp_probs[tuple(sp_index)]
        sp_probs_selected /= sp_probs_selected.sum()  # Normalize to ensure probabilities sum to 1

        # Sample SP's state based on normalized probabilities
        sp_state_idx = np.random.choice(3, p=sp_probs_selected)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]


        # Append data for this entry
        output_data.append({
            'IR_State': ir_state,
            'IRT_State': irt_state,
            'EI_State': ei_state,
            'GEO_State': geo_state,
            'UE_State': ue_state,
            'MS_State': ms_state,
            'IRP_State': irp_state,
            'INF_State': inf_state,
            'GDP_State': gdp_state,
            'INV_State': inv_state,
            'CI_State': ci_state,
            'OIL_State': oil_state,
            'FX_State': fx_state,
            'IRB_State': irb_state,
            'SP_State': sp_state
        })

    # Save the generated dataset
    output_df = pd.DataFrame(output_data)
    output_df.to_csv(filename, index=False)
    print(f"Sample size {sample_size} generated and saved to {filename}")

# Generate data for different sample sizes with varying dependencies
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]
for size in sample_sizes:
    generate_and_save_samples_with_varying_dependencies(size, f'varying_dependencies_data_{size}.csv')

Sample size 50 generated and saved to varying_dependencies_data_50.csv
Sample size 100 generated and saved to varying_dependencies_data_100.csv
Sample size 150 generated and saved to varying_dependencies_data_150.csv
Sample size 200 generated and saved to varying_dependencies_data_200.csv
Sample size 250 generated and saved to varying_dependencies_data_250.csv
Sample size 300 generated and saved to varying_dependencies_data_300.csv
Sample size 350 generated and saved to varying_dependencies_data_350.csv
Sample size 400 generated and saved to varying_dependencies_data_400.csv
Sample size 450 generated and saved to varying_dependencies_data_450.csv
Sample size 500 generated and saved to varying_dependencies_data_500.csv
Sample size 550 generated and saved to varying_dependencies_data_550.csv
Sample size 600 generated and saved to varying_dependencies_data_600.csv
Sample size 650 generated and saved to varying_dependencies_data_650.csv
Sample size 700 generated and saved to varying_depend

# LBN Dense BIC & Entropy

In [22]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000,
                2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'varying_dependencies_data_{sample_size}.csv'  # Update file name to match the data generation
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for each of the 15 nodes
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    ms_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    irp_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    inf_map = {'deflation': 0, 'stable': 1, 'inflation': 2}
    gdp_map = {'low': 0, 'medium': 1, 'high': 2}
    inv_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ci_map = {'low': 0, 'medium': 1, 'high': 2}
    oil_map = {'low': 0, 'medium': 1, 'high': 2}
    fx_map = {'weak': 0, 'stable': 1, 'strong': 2}
    irb_map = {'loose': 0, 'neutral': 1, 'tight': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    # Encode each node in the DataFrame
    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['MS_encoded'] = df['MS_State'].map(ms_map)
    df['IRP_encoded'] = df['IRP_State'].map(irp_map)
    df['INF_encoded'] = df['INF_State'].map(inf_map)
    df['GDP_encoded'] = df['GDP_State'].map(gdp_map)
    df['INV_encoded'] = df['INV_State'].map(inv_map)
    df['CI_encoded'] = df['CI_State'].map(ci_map)
    df['OIL_encoded'] = df['OIL_State'].map(oil_map)
    df['FX_encoded'] = df['FX_State'].map(fx_map)
    df['IRB_encoded'] = df['IRB_State'].map(irb_map)
    df['SP_encoded'] = df['SP_State'].map(sp_map)  # Changed from 'Chosen_SP_State' to 'SP_State'

    # Split the data into training and test sets
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded', 'MS_encoded',
            'IRP_encoded', 'INF_encoded', 'GDP_encoded', 'INV_encoded', 'CI_encoded',
            'OIL_encoded', 'FX_encoded', 'IRB_encoded']]
    y = df['SP_encoded']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

    # Concatenate X_train and y_train to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)
    df_test = pd.concat([X_test, y_test], axis=1)

    # Define the sparse structure with restricted priors to help reduce overfitting and allow the model to learn freely
    priors = [
        ('IR_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with restricted priors on the training data
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Fit parameters using BayesianEstimator on the training data
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=8)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # Filter X_test to include only nodes in the learned structure, excluding SP_encoded
    learned_nodes = [node for node in best_model.nodes() if node != 'SP_encoded']
    X_test_filtered = X_test[learned_nodes]

    # Predict on the filtered test data and calculate K-L divergence
    try:
        # Generate predictions for SP_encoded based on the learned model
        inference = best_model.predict(X_test_filtered)
        predicted_probabilities = inference['SP_encoded']

        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_sparse_dynamic.csv', index=False)

print("\nAll results have been saved.")


Processing sample size: 50


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/3 [00:00<?, ?it/s]

K-L Divergence: 0.1715
Standard Deviation: 0.1963

Processing sample size: 100


...

K-L Divergence: 0.0000
Standard Deviation: 0.0027

Processing sample size: 20000


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/3 [00:00<?, ?it/s]

K-L Divergence: 7.0700
Standard Deviation: 0.2734

All results have been saved.
